In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.formula.api import ols
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from math import sqrt

import scipy, matplotlib
from scipy.optimize import curve_fit
from scipy.optimize import differential_evolution
import warnings


mainData = pd.read_csv('clean_filter_dataframe.csv')
sb.set() # set the default Seaborn style for graphics
mainData.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,...,Fighting,Misc,Platform.1,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
count,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,...,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000
mean,2007.369579,0.260723,0.139294,0.037282,0.048949,0.486453,70.496872,28.914995,7.216677,129.731607,...,0.058252,0.055016,0.061704,0.012945,0.075944,0.083495,0.115426,0.041640,0.163970,0.037972
std,4.069156,0.276439,0.183665,0.117220,0.068543,0.465979,12.903368,18.224541,1.381018,426.797953,...,0.234245,0.228036,0.240644,0.113049,0.264937,0.276659,0.319570,0.199786,0.370288,0.191149
min,1985.000000,0.000000,0.000000,0.000000,0.000000,0.010000,21.000000,3.000000,0.500000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2004.000000,0.070000,0.020000,0.000000,0.010000,0.140000,63.000000,15.000000,6.500000,11.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2007.000000,0.160000,0.060000,0.000000,0.020000,0.320000,72.000000,25.000000,7.500000,25.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2010.000000,0.360000,0.190000,0.010000,0.060000,0.690000,80.000000,40.000000,8.200000,77.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2016.000000,1.820000,1.580000,1.490000,1.180000,2.070000,98.000000,106.000000,9.600000,10665.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [2]:
data = mainData.drop(["Name", 'Developer', 'Publisher', 'Rating', 'Platform', 'Genre'],axis=1)
data.head()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,...,Fighting,Misc,Platform.1,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
0,2005.0,1.77,0.12,0.02,0.17,2.07,72.0,21.0,7.8,54.0,...,0,0,0,0,1,0,0,0,0,0
1,2006.0,0.92,0.93,0.00,0.22,2.07,49.0,5.0,5.2,24.0,...,0,0,0,0,0,0,0,1,0,0
2,2007.0,1.35,0.52,0.00,0.19,2.07,73.0,16.0,7.1,27.0,...,0,0,0,0,0,0,0,1,0,0
3,2014.0,0.36,1.38,0.02,0.31,2.07,71.0,85.0,6.1,1571.0,...,0,0,0,0,1,0,0,0,0,0
4,2009.0,1.48,0.39,0.00,0.19,2.06,83.0,68.0,7.9,51.0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
mainData_dummies = data
mainData_dummies.iloc[:,5:].head(5)
import numpy as np# Labels are the values we want to predict
labels = np.array(mainData_dummies['Global_Sales'])# Remove the labels from the features
# axis 1 refers to the columns
mainData_dummies=  mainData_dummies.drop('Global_Sales', axis = 1)# Saving feature names for later use
mainData_dummies = mainData_dummies.drop("NA_Sales", axis = 1)
mainData_dummies = mainData_dummies.drop("EU_Sales", axis = 1)
mainData_dummies = mainData_dummies.drop("JP_Sales", axis = 1)
mainData_dummies = mainData_dummies.drop("Other_Sales", axis = 1)

mainData_list = list(mainData_dummies.columns)# Convert to numpy array
mainData_dummies = np.array(mainData_dummies)
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(mainData_dummies, labels, test_size = 0.20, random_state = 42)

## Ridge

In [4]:
rr = Ridge(alpha=0.01)
rr.fit(X_train, y_train) 

pred_train_rr= rr.predict(X_train)
print("MSE Train:\t", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R square Train: ", r2_score(y_train, pred_train_rr))

pred_test_rr= rr.predict(X_test)
print("MSE Test:\t",np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R square Test:  ", r2_score(y_test, pred_test_rr))

MSE Train:	 0.3940856369213424
R square Train:  0.26564444426524836
MSE Test:	 0.41684718438814305
R square Test:   0.2707121690935347


## Lasso

In [5]:
model_lasso = Lasso(alpha=0.01)
model_lasso.fit(X_train, y_train) 

pred_train_lasso= model_lasso.predict(X_train)
print("MSE Train:\t",np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
print("R square Train: ", r2_score(y_train, pred_train_lasso))

pred_test_lasso= model_lasso.predict(X_test)
print("MSE Test:\t",np.sqrt(mean_squared_error(y_test,pred_test_lasso))) 
print("R square Test:  ",r2_score(y_test, pred_test_lasso))

MSE Train:	 0.406304901482019
R square Train:  0.21939865608790954
MSE Test:	 0.4306892783816393
R square Test:   0.22147360181055498


## Elastic

In [6]:
model_enet = ElasticNet(alpha = 0.01)
model_enet.fit(X_train, y_train) 

pred_train_enet= model_enet.predict(X_train)
print("MSE Train:\t",np.sqrt(mean_squared_error(y_train,pred_train_enet)))
print("R square Train: ", r2_score(y_train, pred_train_enet))

pred_test_enet= model_enet.predict(X_test)
print("MSE Test:\t",np.sqrt(mean_squared_error(y_test,pred_test_enet)))
print("R square Test:  ",r2_score(y_test, pred_test_enet))

MSE Train:	 0.4013529110648841
R square Train:  0.2383104342825162
MSE Test:	 0.42527105591153963
R square Test:   0.24093865948277937
